In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
from sklearn import ensemble   
from sklearn import datasets   
from sklearn.utils import shuffle   
import xgboost as xgb
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500)

In [9]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
warn_log = True
def logrmse(predictions, targets):
    index = predictions > 0
    if (not index.all()):
        if (warn_log == True):
            warn_log = False
            print 'warning: not positive'
    return rmse(np.log(predictions[index]), np.log(targets[index]))
def xgblogrmse(predictions, dtrain):
    return ('logrmse', logrmse(predictions, dtrain.get_label()))

In [94]:
def resolve():
    ###  read the train, test and macro files
    train_df = pd.read_csv("../input/train.csv", parse_dates=['timestamp'])
    test_df = pd.read_csv("../input/test.csv", parse_dates=['timestamp'])
#     macro_df = pd.read_csv("../input/macro.csv", parse_dates=['timestamp'])
#     print(train_df.shape, test_df.shape)
    
#     # combine macro information with train and test
#     train_df = pd.merge(train_df, macro_df, how='left', on='timestamp')
#     test_df = pd.merge(test_df, macro_df, how='left', on='timestamp')
#     print(train_df.shape, test_df.shape)
    
    # undersampling by magic numbers
#     trainsub = train_df[train_df.timestamp < '2015-01-01']
#     trainsub = trainsub[trainsub.product_type=="Investment"]
#     ind_1m = trainsub[trainsub.price_doc <= 1000000].index
#     ind_2m = trainsub[trainsub.price_doc == 2000000].index
#     ind_3m = trainsub[trainsub.price_doc == 3000000].index
#     train_index = set(train_df.index.copy())
#     for ind, gap in zip([ind_1m, ind_2m, ind_3m], [10, 3, 2]):
#         ind_set = set(ind)
#         ind_set_cut = ind.difference(set(ind[::gap]))
#         train_index = train_index.difference(ind_set_cut)
        
    ###  convert categorical variables into numerical variables by label encoding
    objlist = []
    for f in train_df.columns:
        if train_df[f].dtype=='object':
            objlist.append(f)       
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_df[f].values.astype('str')) + list(test_df[f].values.astype('str')))
            train_df[f] = lbl.transform(list(train_df[f].values.astype('str')))
            test_df[f] = lbl.transform(list(test_df[f].values.astype('str')))
            
    # year and month #
    train_df["yearmonth"] = train_df["timestamp"].dt.year*100 + train_df["timestamp"].dt.month
    test_df["yearmonth"] = test_df["timestamp"].dt.year*100 + test_df["timestamp"].dt.month
    # year and week #
    train_df["yearweek"] = train_df["timestamp"].dt.year*100 + train_df["timestamp"].dt.weekofyear
    test_df["yearweek"] = test_df["timestamp"].dt.year*100 + test_df["timestamp"].dt.weekofyear
    # year #
    train_df["year"] = train_df["timestamp"].dt.year
    test_df["year"] = test_df["timestamp"].dt.year
    # month of year #
    train_df["month_of_year"] = train_df["timestamp"].dt.month
    test_df["month_of_year"] = test_df["timestamp"].dt.month
    # week of year #
    train_df["week_of_year"] = train_df["timestamp"].dt.weekofyear
    test_df["week_of_year"] = test_df["timestamp"].dt.weekofyear
    # day of week #
    train_df["day_of_week"] = train_df["timestamp"].dt.weekday
    test_df["day_of_week"] = test_df["timestamp"].dt.weekday

    ### We could potentially add more variables like this. But for now let us start with model building using these additional variables. Let us drop the variables which are not needed in model building.
    train_X = train_df.drop(["id", "timestamp", "price_doc"], axis=1)
    test_X = test_df.drop(["id", "timestamp"] , axis=1)
    # Since our metric is "RMSLE", let us use log of the target variable for model building rather than using the actual target variable.
    # train_y = np.log1p(train_df.price_doc.values)
    train_y =(train_df.price_doc.values)
    
    return df_train['id'].values, train_X.values, train_y, test_X.values

## 3 baseline models

In [73]:
import bruno
reload(bruno)
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}
num_boost_rounds = 420  # From Bruno's original CV, I think
bruno_xgb = XgbModel(xgb_params, 1000)

In [74]:
import gunja
reload(gunja)
xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.6,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}
num_boost_rounds = 422
gunja_xgb = XgbModel(xgb_params, 1000)

In [6]:
import louis
reload(louis)
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}
num_boost_rounds = 385  # This was the CV output, as earlier version shows
louis_xgb = XgbModel(xgb_params, 1)

# rfr

In [106]:
import rfr
reload(rfr)
my_rfr = ensemble.RandomForestRegressor(n_estimators=70,max_depth=12)
my_rfr = rfr.RfrModel(my_rfr)

## my_xgb

In [95]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}
my_xgb = XgbModel(xgb_params, 360)

# stacking

In [3]:
df_train = pd.read_csv('../input/train.csv')
id_train = df_train['id'].values
y = df_train['price_doc'].values

In [107]:
from stacking import *
import stacking
reload(stacking)

<module 'stacking' from 'stacking.pyc'>

In [108]:
class DataResolver(object):
    def __init__(self):
        self.__time = -1
    
    def next(self):
        self.__time = self.__time + 1
        if (self.__time >= 1):
            return resolve()
        return rfr.resolve()

In [109]:
base_models = []
base_models.append(my_rfr)
base_models.append(my_xgb)
# base_models.append(bruno_xgb)
# base_models.append(gunja_xgb)
# base_models.append(louis_xgb)

In [110]:
stacking = Stacking(5, base_models, DataResolver(), logrmse)

In [111]:
s_train, s_test = stacking.fit(id_train, 7662)#38132 30471

model 0 fold 0
train: 0.390918680186 val: 0.474681384288
model 0 fold 1
train: 0.399627562681 val: 0.456041589892
model 0 fold 2
train: 0.392243549053 val: 0.488764661889
model 0 fold 3
train: 0.398487344793 val: 0.46300950829
model 0 fold 4
train: 0.390540030618 val: 0.477908705342
model 1 fold 0
[0]	train-rmse:8.25762e+06	val-rmse:8.06625e+06
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 20 rounds.
[50]	train-rmse:2.52012e+06	val-rmse:2.9177e+06
[100]	train-rmse:2.19308e+06	val-rmse:2.79422e+06
[150]	train-rmse:2.07142e+06	val-rmse:2.76365e+06
[200]	train-rmse:1.98501e+06	val-rmse:2.75784e+06
[250]	train-rmse:1.92054e+06	val-rmse:2.74936e+06
[300]	train-rmse:1.86354e+06	val-rmse:2.74207e+06
[350]	train-rmse:1.81199e+06	val-rmse:2.73803e+06
[359]	train-rmse:1.80142e+06	val-rmse:2.7369e+06
train: 0.427851834006 val: 0.47056800432
model 1 fold 1
[0]	train-rmse:8.20274e+06	val-rmse:8.23046e+06
Multiple ev

In [112]:
pd.DataFrame(s_train).to_csv('../stacking/rfr_xgb/5_timestamp_train.csv', index=False)

In [113]:
pd.DataFrame(s_test).to_csv('../stacking/rfr_xgb/5_timestamp_test.csv', index=False)

In [114]:
y

array([ 5850000,  6000000,  5700000, ...,  6970959, 13500000,  5600000], dtype=int64)

In [115]:
s_train

array([[  5280882.8937919 ,   4887306.        ],
       [  5304731.95568661,   5194107.        ],
       [  5196958.60327571,   4406640.5       ],
       ..., 
       [  5571862.36072391,   5603753.        ],
       [  9755525.6443084 ,  11062858.        ],
       [  6268209.89780055,   6383557.5       ]])

In [116]:
s_test

array([[ 5449211.18650147,  5725944.        ],
       [ 8490259.03188713,  8526298.8       ],
       [ 5884816.00321119,  5800119.1       ],
       ..., 
       [ 4479278.12520038,  5103306.8       ],
       [ 5648953.38157603,  5672406.2       ],
       [ 8691203.66496464,  8730989.9       ]])

In [129]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.6,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(s_train, y)
#dtrain = xgb.DMatrix(s_train[:25000], y[:25000])
dval = xgb.DMatrix(s_train[25000:], y[25000:])
dtest = xgb.DMatrix(s_test)


num_boost_rounds = 1000
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=500, evals={(dtrain,'train')}, verbose_eval=50)
# model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds, evals=[(dtrain,'train'), (dval,'val')], early_stopping_rounds=20, verbose_eval=20)

y_predict = model.predict(dtest)

[0]	train-rmse:8.19298e+06
[50]	train-rmse:2.48737e+06
[100]	train-rmse:2.31072e+06
[150]	train-rmse:2.23852e+06
[200]	train-rmse:2.18585e+06
[250]	train-rmse:2.14950e+06
[300]	train-rmse:2.11252e+06
[350]	train-rmse:2.07898e+06
[400]	train-rmse:2.04908e+06
[450]	train-rmse:2.02056e+06
[499]	train-rmse:1.99581e+06


In [122]:
df_test = pd.read_csv('../input/test.csv')
id_test = df_test.id

In [130]:
stacking_output = pd.DataFrame({'id': id_test, 'price_doc': y_predict })

In [131]:
stacking_output.to_csv('../stacking.csv', index=False)